#### Configure the Environment

In [ ]:
# Set project name and site
export OS_REGION_NAME=CHI@UC
export OS_PROJECT_NAME=CH-822154

# Define instance variables
NODE_TYPE=compute_haswell
KEYPAIR_NAME="my_chameleon_key"
LEASE_NAME="${USER}Lease"
SERVER_NAME="${USER}Server"

# Define networking variables
NETWORK_NAME=${USER}"Net"
SUBNET_NAME=${USER}"Subnet"
ROUTER_NAME=${USER}"Router"
OF_CONTROLLER_IP=129.114.108.132
OF_CONTROLLER_PORT=6653
DEFAULT_CIDR=192.168.1.0/24

#### Create a Network

In [ ]:
# Create network
openstack network create --max-width 80 \
                         --provider-network-type vlan \
                         --provider-physical-network physnet1 \
                         --description OFController=${OF_CONTROLLER_IP}:${OF_CONTROLLER_PORT} \
                         $NETWORK_NAME

#### Create a Network

In [ ]:
# Obtain public network id.
PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

# Reserve a node and a floating IP on public network.
blazar lease-create \
  --physical-reservation min=1,max=2,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation "resource_type=virtual:floatingip,network_id=$PUBLIC_NETWORK_ID,amount=1" \
  --start-date "$(date +'%Y-%m-%d %H:%M' -d'+1 minute')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$LEASE_NAME"

In [ ]:
# Wait for lease to start
wait_lease "$LEASE_NAME"

#### Configure the Network

In [ ]:
# Create subnet on your new network
openstack subnet create --max-width 80 \
                        --subnet-range $DEFAULT_CIDR \
                        --network $NETWORK_NAME \
                        $SUBNET_NAME
                       
# Create a new router
openstack router create --max-width 80 \
                        $ROUTER_NAME

# Assign your router to a public gateway
openstack router set $ROUTER_NAME --external-gateway "public"

# Add your subnet to your router
openstack router add subnet $ROUTER_NAME $SUBNET_NAME

#### Start a Server on your Network

In [ ]:
# Create a new server using your reservation
openstack server create --max-width 80 \
                        --key-name $KEYPAIR_NAME \
                        $(lease_server_create_default_args "$LEASE_NAME") \
                        $SERVER_NAME

#### Associate the Floating IP

In [ ]:
# Find the unused floating IP you reserved
FLOATING_IP=$(lease_list_floating_ips "$LEASE_NAME" | head -n1)

# Add your floating IP to your server
openstack server add floating ip "$SERVER_NAME" "$FLOATING_IP" \
  && echo "Attached floating ip $FLOATING_IP!"

#### Delete Server

In [ ]:
openstack server delete $SERVER_NAME

#### De-configure Network

In [ ]:
openstack router remove subnet $ROUTER_NAME $SUBNET_NAME
openstack router delete $ROUTER_NAME
openstack subnet delete $SUBNET_NAME
openstack network delete $NETWORK_NAME

#### Release Lease

In [ ]:
blazar lease-delete $LEASE_NAME